In [ ]:
import numpy as np
from matplotlib import pyplot as plt

from data_generation import tbp_energy_calculations, Three_body_2D_Rick
from data_generation import tbp_util

In [ ]:
# todo read from config file
G = 1.0
M = np.array([1.0, 1.0, 1.0])

In [ ]:
def visualize_dataset(x, y, vx, vy, down_sample_factor=100):
    # down sample using slicing [start:stop:step]
    x = x[::down_sample_factor]
    y = y[::down_sample_factor]
    vx = vx[::down_sample_factor]
    vy = vy[::down_sample_factor]

    energy = np.ndarray(dtype=float, shape=(x.shape[0], 3))
    for i in range(energy.shape[0]):
        locations = np.array([x[i], y[i]]).T
        velocities = np.array([vx[i], vy[i]]).T

        potential_energy, kinetic_energy, system_energy = \
            tbp_energy_calculations.energy_values(G, M, locations, velocities)
        energy[i,] = [potential_energy, kinetic_energy, system_energy]
    # energy = pd.DataFrame(energy, columns = ["potential_energy", "kinetic_energy", "system_energy"])
    plt.figure()
    plt.violinplot(energy)
    plt.boxplot(energy)
    plt.show()

    plt.figure()
    plt.plot(energy.T[0])
    plt.plot(energy.T[1])
    plt.plot(energy.T[2])
    plt.legend(["potential_energy", "kinetic_energy", "system_energy"])
    plt.show()

    Three_body_2D_Rick.plot(x, y)


In [ ]:
for dataset, x, y, vx, vy in tbp_util.load_datasets(load_successful=False, load_unsuccessful=True, limit=100):
    visualize_dataset(x, y, vx, vy)

# visualize_dataset(*tbp_util.load_dataset("4"))

In [ ]:
x, y, vx, vy = tbp_util.load_dataset("4")
np.random.choice(vx[0:10,0], 10, replace=False )

In [ ]:
from tensorflow import keras

keras.backend.set_floatx('float64')
keras.backend.floatx()

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(64, activation=keras.activations.relu, input_shape=[12]),
    keras.layers.Dense(64, activation=keras.activations.relu),
    keras.layers.Dense(64, activation=keras.activations.relu),
    keras.layers.Dense(64, activation=keras.activations.relu),
    keras.layers.Dense(64, activation=keras.activations.relu),
    keras.layers.Dense(64, activation=keras.activations.relu),
    keras.layers.Dense(64, activation=keras.activations.relu),
    keras.layers.Dense(12, activation=keras.activations.linear)
])
model.compile(
    keras.optimizers.Adam(learning_rate=0.00001),
    loss='mae',
    metrics=['mae', 'mse']
)

In [ ]:
x_train = np.ndarray((0, 12), dtype=np.float64)
y_train = np.ndarray((0, 12), dtype=np.float64)

for dataset, x, y, vx, vy in tbp_util.load_datasets(limit=100):
    input_data = np.column_stack((x, y, vx, vy))
    output_data = input_data.copy()

    input_data = input_data[:-500:100, ]  # delete last 10
    output_data = output_data[500::100, ]  # delete first 10

    x_train = np.concatenate((x_train, input_data))
    y_train = np.concatenate((y_train, output_data))


In [ ]:
x_train.shape

In [ ]:
tbp_util.load_dataset("4")[0].shape

In [ ]:
history = model.fit(
    x_train, y_train,
    batch_size=256,
    epochs=1024,
    validation_split=0.1
)

In [ ]:
x_train[0,]
x_train.shape

In [ ]:
import math
limit = 1500
y_pred = np.zeros((limit, 12), dtype=np.float64)
y_pred[0,] = x_train[219000,]

for i in range(limit - 1):
    prediction = model(y_pred[i,].reshape(1, 12), training=False).numpy()
    y_pred[i + 1,] = prediction

In [ ]:
x, y, vx, vy = np.hsplit(y_pred, 4)
# x, y, vx, vy = np.hsplit(x_train[:25000], 4)
visualize_dataset(x, y, vx, vy, down_sample_factor=1)

In [ ]:
y_pred